In [ ]:
from pprint import pprint

In [ ]:
from pydantic import BaseModel

class Country(BaseModel):
    name: str
    answer: str


In [ ]:
f = open("./.api-key", "r")
API_KEY = f.read()
print(API_KEY)
f.close()

In [ ]:
from openai import OpenAI, pydantic_function_tool

client = OpenAI(api_key = "API_KEY")

In [ ]:
system_content = "あなたは優秀な歴史学者であり、地政学者です。\
    ユーザーの質問に対して、G20の各国・地域について、それぞれ簡潔に答えてください。\
    必ず「フランス、アメリカ、イギリス、ドイツ、日本、イタリア、カナダ、EU、アルゼンチン、オーストラリア、ブラジル、中国、インド、インドネシア、メキシコ、韓国、ロシア、サウジアラビア、南アフリカ、トルコ、AU」のそれぞれについて説明してください。\
    回答できない場合は「分からない」という説明にしてください。"
query = "主要産業"
completion = client.beta.chat.completions.parse(
    model="gpt-4o",
    messages=[
    {
        "role": "system",
        "content": system_content,
    },
    {
        "role": "user",
        "content": "{}について教えてください。".format(query),
    },
    ],
    tools=[pydantic_function_tool(Country)],
)


In [ ]:
pprint(completion.choices[0].message.tool_calls)

In [ ]:
print(len(completion.choices[0].message.tool_calls))
print(type(completion.choices[0].message))
print(type(completion.choices[0].message.tool_calls))


In [ ]:
texts = list(
  map(
    lambda x: x.function.parsed_arguments.answer,
    completion.choices[0].message.tool_calls,
  )
)

print(texts)

In [ ]:
embedding = client.embeddings.create(input=texts, model="text-embedding-3-small", dimensions=512)
vectors = list(map(lambda x: x.embedding, embedding.data))
pprint(vectors)

In [ ]:
import umap

#上手く行かなかったら以下でリトライ
#import umap.umap_ as umap

reducer = umap.UMAP(n_components=3)
res = reducer.fit_transform(vectors)
print(res)


In [ ]:
import matplotlib.pyplot as plt
import matplotlib_fontja

fig = plt.figure()
ax = fig.add_subplot(projection='3d')
ax.scatter(res[:, 0], res[:, 1], res[:, 2], color='green')

# 各点にラベルを表示
countries = list(map(lambda x: x.function.parsed_arguments.name, completion.choices[0].message.tool_calls))
for i, name in enumerate(countries):
    ax.text(res[i, 0], res[i, 1], res[i, 2], name, fontsize=8)

plt.show()

# # UMAPの結果を2次元プロット
# plt.scatter(res[:, 0], res[:, 1], s=5, c='blue')

# # 各点にラベルを表示
# countries = list(map(lambda x: x.function.parsed_arguments.name, completion.choices[0].message.tool_calls))
# for i, name in enumerate(countries):
#     plt.annotate(name, (res[i, 0], res[i, 1]), fontsize=8)

plt.show()
